In [10]:
import logging
import math
import sys
import os

from jupylet.sprite import Sprite
from jupylet.label import Label
from jupylet.app import App
import numpy as np
import glm  # 確保已安裝 PyGLM

# 設置日誌記錄
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 初始化應用
app = App(width=600, height=600)

# 定義迷宮（1 表示牆，0 表示可通行）
maze = np.array([
    [1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 1, 0, 1, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 1, 1, 1, 1, 1],
])

# 定義迷宮格子大小（像素）
cell_size = 100  # 每個格子的寬度和高度

# 創建玩家
player = Sprite('images.png', x=100, y=100)  # 確保 'images.png' 位於正確路徑
player.scale = glm.vec3(0.5, 0.5, 1.0)  # 根據需要調整縮放比例

# 創建牆壁 Sprites 列表，使用 'images.png' 作為牆壁圖片
wall_sprites = []
for y, row in enumerate(maze):
    for x, cell in enumerate(row):
        if cell == 1:
            wall = Sprite(
                'images.png',                           # 牆壁圖片
                x=x * cell_size,                        # 位置 x
                y=(app.height - (y + 1) * cell_size),    # 位置 y（將迷宮 y 軸反轉以匹配屏幕 y 軸）
                scale=glm.vec3(0.5, 0.5, 1.0)            # 縮放至 50x50（假設 'images.png' 原始大小為 100x100）
            )
            wall_sprites.append(wall)

# 創建標籤（可選）
title_label = Label('迷宮遊戲', color='cyan', font_size=32, x=10, y=app.height - 40)

# 創建座標標籤，置於右上角
coord_label = Label('玩家座標: (100, 100)', color='yellow', font_size=24, 
                    x=app.width - 200, y=app.height - 30)  # 調整 x 以適應文字寬度

# 定義一個函數來檢查目標位置是否可通行
def is_passable(x, y):
    """
    檢查給定的 (x, y) 位置是否在迷宮中且可通行。
    """
    maze_col = int(x / cell_size)
    maze_row = int((app.height - y) / cell_size)
    if 0 <= maze_row < maze.shape[0] and 0 <= maze_col < maze.shape[1]:
        return maze[maze_row][maze_col] == 0
    return False

# 定義一個函數來更新座標標籤
def update_coord_label():
    """
    更新座標標籤以顯示玩家的當前位置。
    """
    coord_text = f'玩家座標: ({int(player.x)}, {int(player.y)})'
    coord_label.text = coord_text
    logger.info(f"更新座標標籤: {coord_text}")

# 處理滑鼠移動事件，讓玩家跟隨滑鼠位置
@app.event
def mouse_position_event(x, y, dx, dy):
    """
    當滑鼠位置改變時觸發此事件。
    """
    logger.info(f"滑鼠移動到: x={x}, y={y}, dx={dx}, dy={dy}")
    # 檢查新位置是否可通行
    if is_passable(x, y):
        player.x = x
        player.y = y
        logger.info(f"玩家移動到: x={player.x}, y={player.y}")
        update_coord_label()
    else:
        logger.info("嘗試移動玩家到牆壁位置。")

# 繪製迷宮和玩家
@app.event
def render(ct, dt):
    """
    每幀渲染時觸發此事件。
    """
    app.window.clear()
    for wall in wall_sprites:
        wall.draw()
    player.draw()
    title_label.draw()
    coord_label.draw()

# 初始化座標標籤
update_coord_label()

if __name__ == "__main__":
    app.run()


In [11]:
app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x05\x03\x0…